In [69]:
import torch
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [70]:
df = pd.read_csv("./AMZN.csv")
df["Date"] = pd.to_datetime(df["Date"])
df.set_index(df["Date"])

data = df[[i]"Date", "Close"]]

SyntaxError: unmatched ']' (1227406284.py, line 5)

In [71]:
df = pd.read_csv("./AMZN.csv")
df["Date"] = pd.to_datetime(df["Date"])
df.set_index(df["Date"])
data = df[["Date", "Close"]]

def data_with_context(df, context_size):

    data.set_index("Date", inplace=True)
    for i in range(context_size):
        data[f"Close t-{i+1}"] = data["Close"].shift(i+1)

    final_df = data.iloc[context_size:]
    return final_df


data = data_with_context(df, 7)


/tmp/ipykernel_3022/2107889153.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f"Close t-{i+1}"] = data["Close"].shift(i+1)
/tmp/ipykernel_3022/2107889153.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f"Close t-{i+1}"] = data["Close"].shift(i+1)
/tmp/ipykernel_3022/2107889153.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [72]:
data

,Close,Close t-1,Close t-2,Close t-3,Close t-4,Close t-5,Close t-6,Close t-7
Date,,,,,,,,
1997-05-27,0.079167,0.075000,0.069792,0.071354,0.081771,0.085417,0.086458,0.097917
1997-05-28,0.076563,0.079167,0.075000,0.069792,0.071354,0.081771,0.085417,0.086458
1997-05-29,0.075260,0.076563,0.079167,0.075000,0.069792,0.071354,0.081771,0.085417
1997-05-30,0.075000,0.075260,0.076563,0.079167,0.075000,0.069792,0.071354,0.081771
1997-06-02,0.075521,0.075000,0.075260,0.076563,0.079167,0.075000,0.069792,0.071354
...,...,...,...,...,...,...,...,...
2023-03-30,102.000000,100.250000,97.239998,98.040001,98.129997,98.709999,98.699997,100.610001
2023-03-31,103.290001,102.000000,100.250000,97.239998,98.040001,98.129997,98.709999,98.699997
2023-04-03,102.410004,103.290001,102.000000,100.250000,97.239998,98.040001,98.129997,98.709999


In [77]:
from copy import deepcopy as dc
data_numpy = data.to_numpy()
#print(data_numpy) # first element of each row is the label, the following numbers are the context
#print(data_numpy.shape)

#in the df it is ordered by recent --> former
#But the lstm takes in from the last inputs to the most recent ones
data_numpy = dc(np.flip(data_numpy, axis=1))
print(data_numpy)



[[9.79170000e-02 8.64580000e-02 8.54170000e-02 ... 6.97920000e-02
  7.50000000e-02 7.91670000e-02]
 [8.64580000e-02 8.54170000e-02 8.17710000e-02 ... 7.50000000e-02
  7.91670000e-02 7.65630000e-02]
 [8.54170000e-02 8.17710000e-02 7.13540000e-02 ... 7.91670000e-02
  7.65630000e-02 7.52600000e-02]
 ...
 [9.87099990e+01 9.81299970e+01 9.80400010e+01 ... 1.02000000e+02
  1.03290001e+02 1.02410004e+02]
 [9.81299970e+01 9.80400010e+01 9.72399980e+01 ... 1.03290001e+02
  1.02410004e+02 1.03949997e+02]
 [9.80400010e+01 9.72399980e+01 1.00250000e+02 ... 1.02410004e+02
  1.03949997e+02 1.01099998e+02]]


In [91]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
data_numpy = scaler.fit_transform(data_numpy)
train_percent = 0.95
test_percent = 0.05

train_split = int(len(data_numpy)*train_percent)

train_data = data_numpy[:train_split]
test_data = data_numpy[train_split:]


In [208]:
train_inputs = train_data[:, :-1]
train_labels = train_data[:, -1].reshape(train_inputs.shape[0], 1) #1 label per row
print(train_inputs)
print(train_inputs.shape)

print(train_labels)
print(train_labels.shape)

[[-0.99969839 -0.99982128 -0.99983244 ... -0.99998325 -1.
  -0.99995531]
 [-0.99982128 -0.99983244 -0.99987154 ... -1.         -0.99994415
  -0.99991063]
 [-0.99983244 -0.99987154 -0.99998325 ... -0.99994415 -0.99989946
  -0.99993855]
 ...
 [ 0.81682649  0.83697653  0.88840802 ...  0.86703009  0.84602214
   0.81766197]
 [ 0.83697653  0.88840802  0.88833832 ...  0.84602214  0.81766298
   0.81255742]
 [ 0.88840802  0.88833832  0.86703009 ...  0.81766298  0.81255847
   0.85784967]]
(6183, 7)
[[-0.99991063]
 [-0.99993855]
 [-0.99995253]
 ...
 [ 0.81255742]
 [ 0.85784967]
 [ 0.81019275]]
(6183, 1)


In [209]:
test_inputs = test_data[:, :-1]
test_labels = test_data[:, -1].reshape(test_inputs.shape[0], 1) #1 label per row
print(test_inputs)
print(test_inputs.shape)

print(test_labels)
print(test_labels.shape)

[[0.88833832 0.86703009 0.84602214 ... 0.81255847 0.85785047 0.81019275]
 [0.86703009 0.84602214 0.81766298 ... 0.85785047 0.81019381 0.82248768]
 [0.84602214 0.81766298 0.81255847 ... 0.81019381 0.82248867 0.79097548]
 ...
 [0.05779984 0.05158    0.0506149  ... 0.07431453 0.09308121 0.10690997]
 [0.05158    0.0506149  0.04203581 ... 0.09308121 0.10691495 0.09747299]
 [0.0506149  0.04203581 0.07431453 ... 0.10691495 0.09747802 0.11398769]]
(326, 7)
[[ 0.82248768]
 [ 0.79097548]
 [ 0.82677192]
 [ 0.83342069]
 [ 0.83375842]
 [ 0.81875569]
 [ 0.82938842]
 [ 0.81373165]
 [ 0.80776378]
 [ 0.78709344]
 [ 0.82663775]
 [ 0.79572625]
 [ 0.76178508]
 [ 0.74995667]
 [ 0.74244999]
 [ 0.73099679]
 [ 0.77256263]
 [ 0.77090044]
 [ 0.72808003]
 [ 0.73798885]
 [ 0.70345255]
 [ 0.67537207]
 [ 0.62570442]
 [ 0.52892686]
 [ 0.54931297]
 [ 0.50043337]
 [ 0.48849239]
 [ 0.49669615]
 [ 0.54324315]
 [ 0.60324868]
 [ 0.62062131]
 [ 0.6143908 ]
 [ 0.48820293]
 [ 0.68974743]
 [ 0.69292175]
 [ 0.73021937]
 [ 0.72

In [222]:
train_inputs_tensor = torch.tensor(train_inputs)
train_labels_tensor = torch.tensor(train_labels)

test_inputs_tensor = torch.tensor(test_inputs)
test_labels_tensor = torch.tensor(test_labels)



In [223]:

#train_inputs = torch.unsqueeze(train_inputs, dim=1)
#test_inputs = torch.unsqueeze(test_inputs, dim=1)


train_inputs_tensor = train_inputs_tensor.unsqueeze(-1)
#train_labels_tensor = train_inputs_tensor.unsqueeze(-1)

test_inputs_tensor = test_inputs_tensor.unsqueeze(-1)

print(train_inputs_tensor.shape)
print(test_inputs_tensor.shape)

train_dataset = torch.utils.data.TensorDataset(train_inputs_tensor, train_labels_tensor)
train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

torch.Size([6183, 7, 1])
torch.Size([326, 7, 1])


In [224]:
print(next(iter(train_dataloader))[0].shape)

torch.Size([32, 7, 1])
